In [12]:
import pandas as pd

In [13]:
encodings_to_try = ['utf-8', 'latin-1', 'ISO-8859-1']

for encoding in encodings_to_try:
    try:
        df = pd.read_csv('AI EarthHack Dataset.csv', encoding=encoding)
        break  
    except UnicodeDecodeError:
        continue 

In [14]:
# check all the data is there 
len(df)

# look for null values 
df.isnull().sum()

# drop null values
df = df.dropna()


In [15]:
df.head()

,id,problem,solution
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m..."
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single..."
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa..."
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...


In [16]:
# use chain of thought prompting 
# feed the problem first 
test_prob = df['problem'][0]
test_sol = df['solution'][0]

print(test_prob)

The construction industry is indubitably one of the significant contributors to global waste, contributing approximately 1.3 billion tons of waste annually, exerting significant pressure on our landfills and natural resources. Traditional construction methods entail single-use designs that require frequent demolitions, leading to resource depletion and wastage.   


In [17]:
# find the longest problem 

long_prob = df['problem'][0]

for problem in df['problem']:
    if len(problem) > len(long_prob):
        long_prob = problem

len(long_prob)  


4627

In [18]:
# find the longest solution 

long_sol = df['solution'][0]

for solution in df['solution']:
    if len(solution) > len(long_sol):
        long_sol = solution

len(long_sol)  

7617

In [19]:
# !pip install -q transformers einops accelerate langchain bitsandbytes
# !pip install sentence_transformers
# !pip install llama-index
# pip install accelerate
# pip install bitsandbytes

In [20]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext
from llama_index.prompts.prompts import SimpleInputPrompt
from transformers import AutoTokenizer,AutoModelForCausalLM

import transformers
import torch

In [21]:
def load_models(
                 model_dir:str = "Model_dir_path",
                 model_name:str = "tiiuae/falcon-7b-instruct",
                 auth_token:str = "acess_token_from_huggingface_if_needed",
                 load_in_8bit:bool = True

               ):


    try :
        print("Cheking and if Model and tokenizer is in Memory...")
        falcon_tokenizer = AutoTokenizer.from_pretrained(model_dir+"/"+"tokenizer")
        falcon_model = FalconForCausalLM.from_pretrained(model_dir+"/"+"model")
        print("Model and tokenizer loaded from memory...")

    except :
        print("Couldn't find Model in memory so loading from Huggingface...")
        falcon_tokenizer = AutoTokenizer.from_pretrained(model_name,use_auth_token = auth_token)
        falcon_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=load_in_8bit,  use_auth_token=auth_token)
        print("Saving Model and Tokenizer in Memory for future refference...")

        #os.makedirs(model_dir)

        tokenizer_folder = "tokenizer"
        tokenizer_path = model_dir+"/"+tokenizer_folder
        #os.makedirs(tokenizer_path)
        falcon_tokenizer.save_pretrained(tokenizer_path)

        model_folder = "model"
        model_path = model_dir+"/"+model_folder
        #os.makedirs(model_path)
        falcon_model.save_pretrained(model_path, from_pt=True)
        print("Tokenizer and model saved...")


    return falcon_tokenizer, falcon_model

In [22]:
falcon_tokenizer,falcon_model = load_models()

Cheking and if Model and tokenizer is in Memory...
Couldn't find Model in memory so loading from Huggingface...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [23]:
pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install accelerate


Note: you may need to restart the kernel to use updated packages.
